In [2]:
import sys
sys.path.append('../data/')

In [3]:
import nltk
from nltk.tokenize import TreebankWordTokenizer, PunktSentenceTokenizer
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger
from nltk.corpus import brown
import re
from pathlib import Path
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
import csv
import spacy
import toolz as tz
import toolz.curried as tzc

In [4]:
nlp = spacy.load('en')

In [5]:
!ls ../data/

labeledTrainData.tsv   testData.tsv
sampleSubmission.csv   unlabeledTrainData.tsv


In [6]:
train_file = Path('../data/labeledTrainData.tsv')
 

In [7]:
f = train_file.open()

In [8]:
records = csv.DictReader(f, delimiter='\t')

## Text Processing

In [11]:
samples = tz.take(10, records)

In [12]:
reviews = tuple(tz.pluck('review', samples))

In [13]:
review = reviews[1]
print(review)

\The Classic War of the Worlds\" by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells' classic book. Mr. Hines succeeds in doing so. I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g. the Spielberg version with Tom Cruise that had only the slightest resemblance to the book. Obviously, everyone looks for different things in a movie. Those who envision themselves as amateur \"critics\" look only to criticize everything they can. Others rate a movie on more important bases,like being entertained, which is why most people never agree with the \"critics\". We enjoyed the effort Mr. Hines put into being faithful to H.G. Wells' classic novel, and we found it to be very entertaining. This made it easy to overlook what the \"critics\" perceive to be its shortcomings."


In [14]:
sents = nltk.sent_tokenize(review)
sents

['\\The Classic War of the Worlds\\" by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells\' classic book.',
 'Mr. Hines succeeds in doing so.',
 'I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g.',
 'the Spielberg version with Tom Cruise that had only the slightest resemblance to the book.',
 'Obviously, everyone looks for different things in a movie.',
 'Those who envision themselves as amateur \\"critics\\" look only to criticize everything they can.',
 'Others rate a movie on more important bases,like being entertained, which is why most people never agree with the \\"critics\\".',
 'We enjoyed the effort Mr. Hines put into being faithful to H.G.',
 "Wells' classic novel, and we found it to be very entertaining.",
 'This made it easy to overlook what the \\"critics\\" perceive to be its shortcomings."']

In [15]:
gold_words = nltk.word_tokenize(review)
gold_words;

In [15]:
sent_regex = re.compile(r'')

In [16]:
help(re)

Help on module re:

NAME
    re - Support for regular expressions (RE).

MODULE REFERENCE
    https://docs.python.org/3.7/library/re
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    This module provides regular expression matching operations similar to
    those found in Perl.  It supports both 8-bit and Unicode strings; both
    the pattern and the strings being processed can contain null bytes and
    characters outside the US ASCII range.
    
    Regular expressions can contain both special and ordinary characters.
    Most ordinary characters, like "A", "a", or "0", are the simplest
    regular expressions; they simply match themselves.  You can
    concatenate ordinary characters, so last mat

In [16]:
words = nltk.regexp_tokenize(review, 
r'''(?x)
\w+(?:[-'.]\w+)* | [A-Z]+(?:\.[A-Z]+)*
''')
words;

In [245]:
diff = set(gold_words).difference(words)
error = len(diff) / len(gold_words)
print(f'error rate: {error}')
print(diff)

error rate: 0.05789473684210526
{'G.', '\\', ',', "'", 'critics\\', 'H.', 'Mr.', '\\The', 'Worlds\\', "''", '.'}


In [254]:
TreebankWordTokenizer?

Init signature: TreebankWordTokenizer()
Docstring:     
The Treebank tokenizer uses regular expressions to tokenize text as in Penn Treebank.
This is the method that is invoked by ``word_tokenize()``.  It assumes that the
text has already been segmented into sentences, e.g. using ``sent_tokenize()``.

This tokenizer performs the following steps:

- split standard contractions, e.g. ``don't`` -> ``do n't`` and ``they'll`` -> ``they 'll``
- treat most punctuation characters as separate tokens
- split off commas and single quotes, when followed by whitespace
- separate periods that appear at the end of line

    >>> from nltk.tokenize import TreebankWordTokenizer
    >>> s = '''Good muffins cost $3.88\nin New York.  Please buy me\ntwo of them.\nThanks.'''
    >>> TreebankWordTokenizer().tokenize(s)
    ['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York.', 'Please', 'buy', 'me', 'two', 'of', 'them.', 'Thanks', '.']
    >>> s = "They'll save and invest more."
    >>> TreebankWordTo

In [255]:
PunktSentenceTokenizer?

Init signature:
PunktSentenceTokenizer(
    train_text=None,
    verbose=False,
    lang_vars=None,
    token_cls=<class 'nltk.tokenize.punkt.PunktToken'>,
)
Docstring:     
A sentence tokenizer which uses an unsupervised algorithm to build
a model for abbreviation words, collocations, and words that start
sentences; and then uses that model to find sentence boundaries.
This approach has been shown to work well for many European
languages.
Init docstring:
train_text can either be the sole training text for this sentence
boundary detector, or can be a PunktParameters object.
File:           ~/.local/share/virtualenvs/imdb_dataset-ZDLglFKy/lib/python3.7/site-packages/nltk/tokenize/punkt.py
Type:           ABCMeta
Subclasses:     


In [17]:
sents_tokenizer = PunktSentenceTokenizer()
words_tokenizer = TreebankWordTokenizer()


In [18]:
tokenize_words = tzc.compose(tzc.mapcat(words_tokenizer.tokenize), sents_tokenizer.tokenize)

In [20]:
tokens = list(tokenize_words(review))

In [21]:
tagged_sents = brown.tagged_sents(categories='editorial', tagset='universal')
length = len(tagged_sents)
print(length)
p = 0.8
train = tagged_sents[: int(length * p)]
test = tagged_sents[int(length * p):]

2997


In [22]:
def evaluate(tagger, test_sents):
    print(tagger.evaluate(test_sents))

In [23]:
t0 = DefaultTagger('NN')
evaluate(t0, test)

t1 = UnigramTagger(train, backoff=t0)
evaluate(t1, test)

t2 = BigramTagger(train, backoff=t1)
evaluate(t2, test)

t3 = nltk.tag.TrigramTagger(train, backoff=t2)
evaluate(t3, test)

0.0
0.8180710969018802
0.8226383496993225
0.8224099870594505


In [24]:
trainer = nltk.tag.HiddenMarkovModelTrainer()
hmm = trainer.train(train)
evaluate(hmm , test)

0.46958970845702974


In [19]:
from collections import Counter

In [20]:
def gender_features(word):
    return {
        'last_letter': word[-2:].lower(),
        'length': len(word),
        'first_letter': word[:2].lower(),
        'most_frequent_letter': Counter(word.lower()).most_common(1)[0][0]
    }

In [21]:
gender_features('Ndil')

{'last_letter': 'il',
 'length': 4,
 'first_letter': 'nd',
 'most_frequent_letter': 'n'}

In [22]:
from nltk.corpus import names as names_corpus
import random 

In [26]:
names = [(name, 'male') for name in names_corpus.words('male.txt')] + [(name, 'female') for name in names_corpus.words('female.txt')]
random.shuffle(names)
len(names)

7944

In [13]:
featuresets = [(gender_features(n), g) for (n, g) in names]
train_set, test_set = featuresets[500:], featuresets[:500]

In [14]:
clf = nltk.NaiveBayesClassifier.train(train_set)

In [15]:
print(clf.classify(gender_features('John')))
print(clf.classify(gender_features('Mollie')))
print(nltk.classify.accuracy(clf, test_set))

male
female
0.766


In [16]:
clf.show_most_informative_features(5)

Most Informative Features
             last_letter = 'na'           female : male   =    101.3 : 1.0
             last_letter = 'la'           female : male   =     75.5 : 1.0
             last_letter = 'ia'           female : male   =     39.9 : 1.0
             last_letter = 'sa'           female : male   =     34.8 : 1.0
             last_letter = 'rd'             male : female =     31.5 : 1.0


In [27]:
import sklearn

In [28]:
train_names = names[1500:]
devtest_names = names[500:1500]
test_names = names[:500]

In [29]:
@tz.curry
def extract_features(feature_function, dataset):
    return [(feature_function(d), l) for (d, l) in dataset]

In [30]:
extract_gender_features = extract_features(gender_features)
train_set = extract_gender_features(train_names)
devtest_set = extract_gender_features(devtest_names)
test_set = extract_gender_features(test_names)

In [62]:
clf = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(clf, devtest_set))

0.816


In [63]:
errors = []
for (n, tag) in devtest_names:
    guess = clf.classify(gender_features(n))
    if guess != tag:
        errors.append((tag, guess, n))

In [64]:
for (tag, guess, name) in sorted(errors):
    print(f'correct={tag} guess={guess} name={name}')

correct=female guess=male name=Abbe
correct=female guess=male name=Abbie
correct=female guess=male name=Abby
correct=female guess=male name=Bambie
correct=female guess=male name=Beilul
correct=female guess=male name=Buffy
correct=female guess=male name=Bunnie
correct=female guess=male name=Cam
correct=female guess=male name=Chad
correct=female guess=male name=Charlot
correct=female guess=male name=Chloe
correct=female guess=male name=Chrysler
correct=female guess=male name=Clary
correct=female guess=male name=Cyb
correct=female guess=male name=Darby
correct=female guess=male name=Darsey
correct=female guess=male name=Debor
correct=female guess=male name=Devi
correct=female guess=male name=Devin
correct=female guess=male name=Dorolice
correct=female guess=male name=Dulce
correct=female guess=male name=Edin
correct=female guess=male name=Em
correct=female guess=male name=Emmy
correct=female guess=male name=Erin
correct=female guess=male name=Esther
correct=female guess=male name=Evvy
cor

In [31]:
from nltk.corpus import movie_reviews

In [32]:
documents = [
    (list(movie_reviews.words(fileid)), category)
    for category in movie_reviews.categories()
    for fileid in movie_reviews.fileids(category)
]

In [24]:
random.shuffle(documents)

In [25]:
len(documents)

2000

In [90]:
stopwords = nltk.corpus.stopwords.words('english')

In [107]:
stemmer = nltk.PorterStemmer()

In [157]:
def filter_stopword(word):
    return word not in stopwords

def process_words(words):
    return tz.pipe(words, tzc.filter(filter_stopword), tzc.map(stemmer.stem))


In [181]:
all_words = list(process_words(movie_reviews.words()))
word_freqs =  Counter(all_words)
# all_words = Counter(w.lower() for w in movie_reviews.words() if w not in stopwords)

In [159]:
word_features = set(w for (w, f) in all_words.most_common(2000))

In [46]:
test = { f'contains({word})': True if word in present else False for word in sd }

In [165]:
from nltk.collocations import BigramCollocationFinder, ngrams
from nltk.metrics import BigramAssocMeasures

In [196]:
def find_bigrams(words, scorer=BigramAssocMeasures.chi_sq, size=200):
    print(words)
    bigram_finder = BigramCollocationFinder.from_words(words)
#     print(type(bigram_finder.score_ngrams(scorer)))
    bigrams = bigram_finder.nbest(scorer, size)
    return bigrams

In [192]:
list(tz.take(10, all_words))

['plot', ':', 'two', 'teen', 'coupl', 'go', 'church', 'parti', ',', 'drink']

In [197]:
find_bigrams()

['plot', ':', 'two', 'teen', 'coupl', 'go', 'church', 'parti', ',', 'drink']


[(',', 'drink'),
 (':', 'two'),
 ('church', 'parti'),
 ('coupl', 'go'),
 ('go', 'church'),
 ('parti', ','),
 ('plot', ':'),
 ('teen', 'coupl'),
 ('two', 'teen')]

In [160]:
def document_features(document):
    document_words = set(process_words(document))
    contained = document_words.intersection(word_features)
    features = { 
        f'contains({word})': True if word in contained else False 
        for word in document_words 
    }
        
    return features

def document_features_bow(document):
    document_words = set(process_words(document))
#     contained = document_words.intersection(word_features)
    features = { 
        f'contains({word})': True if word in document_words else False 
        for word in word_features 
    }
        
    return features

def document_features_nltk(document):
    document_words = set(document)
    features = {}

    for word in word_features:
        features[f'contains({word})'] = word in document_words
    return features

extract_document_features = extract_features(document_features)
extract_document_features_bow = extract_features(document_features_bow)
extract_document_features_nltk = extract_features(document_features_nltk)

In [ ]:
document_features_bow(documents[-1][0]);

In [152]:
%%timeit
extract_document_features(documents[:10])

92 ms ± 399 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [153]:
%%timeit
extract_document_features_bow(documents[:10])

95.8 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [67]:
%%timeit
extract_document_features_nltk(documents[:10])

4.1 s ± 217 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [161]:
%%time
print('train set')
train_set = extract_document_features_bow(documents[200:])
print('devtest set')
devtest_set = extract_document_features_bow(documents[100:200])
print('test set')
test_set = extract_document_features_bow(documents[:100])

train set
devtest set
test set
CPU times: user 17.1 s, sys: 173 ms, total: 17.2 s
Wall time: 17.3 s


In [162]:
clf = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(clf, devtest_set))
print(clf.show_most_informative_features(5))

0.71
Most Informative Features
      contains(outstand) = True              pos : neg    =     10.0 : 1.0
        contains(seagal) = True              neg : pos    =      8.4 : 1.0
         contains(mulan) = True              pos : neg    =      8.1 : 1.0
        contains(poorli) = True              neg : pos    =      6.6 : 1.0
          contains(lame) = True              neg : pos    =      6.5 : 1.0
None


In [130]:
errors = []
for (n, tag) in devtest_set:
    guess = clf.classify(document_features(n))
    if guess != tag:
        errors.append((tag, guess, n))
len(errors)

54

In [139]:
import json

In [ ]:
for i, (tag, guess, name) in enumerate(sorted(errors, key=lambda d: d[:2])):
    print(f'correct={tag} guess={guess} name={json.dumps(name, indent=2)}')
    if i >= 5:
        break

In [33]:
from nltk.corpus import brown

In [38]:
suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1

In [42]:
print(list(tz.pluck(0, suffix_fdist.most_common(100))))

['e', ',', '.', 's', 'd', 't', 'he', 'n', 'a', 'of', 'the', 'y', 'r', 'to', 'in', 'f', 'o', 'ed', 'nd', 'is', 'on', 'l', 'g', 'and', 'ng', 'er', 'as', 'ing', 'h', 'at', 'es', 'or', 're', 'it', '``', 'an', "''", 'm', ';', 'i', 'ly', 'ion', 'en', 'al', '?', 'nt', 'be', 'hat', 'st', 'his', 'th', 'll', 'le', 'ce', 'by', 'ts', 'me', 've', "'", 'se', 'ut', 'was', 'for', 'ent', 'ch', 'k', 'w', 'ld', '`', 'rs', 'ted', 'ere', 'her', 'ne', 'ns', 'ith', 'ad', 'ry', ')', '(', 'te', '--', 'ay', 'ty', 'ot', 'p', 'nce', "'s", 'ter', 'om', 'ss', ':', 'we', 'are', 'c', 'ers', 'uld', 'had', 'so', 'ey']


In [45]:
def pos_features(sentence, i):
    features = {
        'suffix(1)': sentence[i][-1:],
        'suffix(2)': sentence[i][-2:],
        'suffix(3)': sentence[i][-3:]
    }
    
    if i == 0:
        features['prev-word'] = '<START>'
    else: 
        features['prev-word'] = sentence[i-1]
    return features


In [46]:
pos_features(brown.sents()[0], 8)

{'suffix(1)': 'n', 'suffix(2)': 'on', 'suffix(3)': 'ion', 'prev-word': 'an'}

In [48]:
tagged_sents = brown.tagged_sents(categories='news')
featuresets = []
for tagged_sent in tagged_sents:
    untagged_sent = nltk.untag(tagged_sent)
    for i, (word, tag) in enumerate(tagged_sent):
        featuresets.append((pos_features(untagged_sent, i), tag))

In [49]:
featuresets[0]

({'suffix(1)': 'e',
  'suffix(2)': 'he',
  'suffix(3)': 'The',
  'prev-word': '<START>'},
 'AT')

In [51]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
clf = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(clf, test_set)

0.7891596220785678

In [53]:
def pos_feature_history(sentence, i, history):
    features = {
        'suffix(1)': sentence[i][-1:],
        'suffix(2)': sentence[i][-2:],
        'suffix(3)': sentence[i][-3:]
    }
    
    if i == 0:
        features['prev-word'] = '<START>'
        features['prev-tag'] = '<START>'
    else:
        features['prev-word'] = sentence[i-1]
        features['prev-tag'] = history[i-1]   
    return features


In [66]:
class ConsecutivePosTagger(nltk.TaggerI):
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in tagged_sents:
            untagged_sent = nltk.untag(tagged_sent)
            history = []
            for i, (_, tag) in enumerate(tagged_sent):
                featuresets = pos_feature_history(untagged_sent, i, history)
                history.append(tag)
                train_set.append((featuresets, tag))
        self.clf = nltk.NaiveBayesClassifier.train(train_set)
        
    def tag(self, sentence):
        tagged_sent = []
        history = []
        for i, word in enumerate(sentence):
            featuresets = pos_feature_history(sentence, i, history)
            tag = self.clf.classify(featuresets)
            history.append(tag)
            tagged_sent.append((word, tag))
        return tagged_sent

In [67]:
size = int(len(tagged_sents) * 0.1)
train_sents, test_sents = tagged_sents[size:], tagged_sents[:size]
tagger = ConsecutivePosTagger(train_sents)


In [69]:
tagger.tag(nltk.untag(test_sents[0]))

[('The', 'AT'),
 ('Fulton', 'NN'),
 ('County', 'NN-TL'),
 ('Grand', 'CC'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]

In [70]:
test_sents[0]

[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]

In [71]:
import pandas as pd

In [173]:
from sklearn import feature_extraction, preprocessing, pipeline
from sklearn import naive_bayes, linear_model, tree
from concurrent.futures import ProcessPoolExecutor

In [174]:
def create_tagged_dataset(tagged_sents):
    X, y = [], []
    for tagged_sent in tagged_sents:
        untagged_sent = nltk.untag(tagged_sent)
        history = []
        for i, (_, tag) in enumerate(tagged_sent):
            featuresets = pos_feature_history(untagged_sent, i, history)
            history.append(tag)
            X.append(featuresets)
            y.append(tag)
    return X, y

def predict_tags(estimator, labelizer, sentence):
    history = []
    for i, word in enumerate(sentence):
        features = pos_feature_history(sentence, i, history)
        label = estimator.predict(features)
        tag = labelizer.inverse_transform(label)
        history.append(tag[0])
    return list(zip(sentence, history))
        
        
class ConsecutivePosTagEstimator(nltk.TaggerI):
    def __init__(self, clf, train_sents):
        self.estimator = pipeline.make_pipeline(feature_extraction.DictVectorizer(), clf)
        features, tags = create_tagged_dataset(train_sents)
        self.labelizer = preprocessing.LabelEncoder()
        y = self.labelizer.fit_transform(tags)
        self.estimator.fit(features, y)
        
    def tag(self, sentence):
        return predict_tags(self.estimator, self.labelizer, sentence)
    
    def tag_sents(self, sentences):
        with ProcessPoolExecutor() as executor:
            tagged_sents = executor.map(self.tag, sentences)
        return tagged_sents
            

In [143]:
test_p, devtest_p = 0.1, 0.1
test_i, devtest_i = int(len(tagged_sents) * test_p), int(len(tagged_sents) * (test_p + devtest_p))
test_sents, devtest_sents, train_sents = tagged_sents[:test_i], tagged_sents[test_i:devtest_i], tagged_sents[devtest_i:] 

In [150]:
features_train, tags_train = create_tagged_dataset(train_sents)

In [147]:
features_train[:3]

[{'suffix(1)': 'e',
  'suffix(2)': 'He',
  'suffix(3)': 'He',
  'prev-word': '<START>',
  'prev-tag': '<START>'},
 {'suffix(1)': 'd',
  'suffix(2)': 'ed',
  'suffix(3)': 'ted',
  'prev-word': 'He',
  'prev-tag': 'PPS'},
 {'suffix(1)': 't',
  'suffix(2)': 'st',
  'suffix(3)': 'ast',
  'prev-word': 'cited',
  'prev-tag': 'VBD'}]

In [151]:
labelizer = preprocessing.LabelEncoder()
y_train = labelizer.fit_transform(tags_train)

In [116]:
# encoder = preprocessing.OneHotEncoder()
# X_train = encoder.fit_transform(features_train)

In [117]:
clf = naive_bayes.BernoulliNB()
# clf.fit(X, y)

In [152]:
estimator = pipeline.make_pipeline(feature_extraction.DictVectorizer(), clf)
estimator.fit(features_train, y_train)

Pipeline(memory=None,
     steps=[('dictvectorizer', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=True)), ('bernoullinb', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True))])

In [153]:
labelizer.inverse_transform([30])

array(['BEDZ-HL'], dtype='<U11')

In [158]:
predict_tags(estimator, labelizer, nltk.untag(devtest_sents[0]))

[('He', 'AT'),
 ('assured', 'VBD'),
 ('Mr.', 'NP'),
 ('Martinelli', 'NP'),
 ('and', 'CC'),
 ('the', 'AT'),
 ('council', 'NN'),
 ('that', 'IN'),
 ('he', 'AT'),
 ('would', 'NN'),
 ('study', 'NN'),
 ('the', 'AT'),
 ('correct', 'NN'),
 ('method', 'NN'),
 ('and', 'CC'),
 ('report', 'NN'),
 ('back', 'NN'),
 ('to', 'IN'),
 ('the', 'AT'),
 ('council', 'NN'),
 ('as', 'NNS'),
 ('soon', 'NN'),
 ('as', 'CS'),
 ('possible', 'JJ'),
 ('.', '.')]

In [177]:
%%time
naive_bayes_tagger = ConsecutivePosTagEstimator(naive_bayes.BernoulliNB(), train_sents)

CPU times: user 1.01 s, sys: 152 ms, total: 1.16 s
Wall time: 1.17 s


In [178]:
%%time
naive_bayes_tagger.evaluate(devtest_sents)

CPU times: user 18.5 s, sys: 19.9 s, total: 38.4 s
Wall time: 5min 42s


0.601364623177918